In [ ]:
# Loading libraries
library(Gviz)
library(GenomicRanges)
library(rtracklayer)
library(GenomicAlignments)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(org.Hs.eg.db)
library(BiocManager)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(Biostrings)
library(KEGGREST)
library(AnnotationDbi)
library(GenomicAlignments)
library(rtracklayer)
library(biomaRt)
library(GenomicFeatures)
library(BSgenome)
library(plotgardener)

In [ ]:
# Define the region of interest 
chrom <- "chr7"
start <- 18086949
end <- 19002416

In [ ]:
# Load GTF file
gtf_file <- "gencode.v44.basic.annotation.gtf"
gtf_data <- import(gtf_file)

# Filter gtf_data for the specific chromosome and region
subset_gtf_data <- gtf_data[seqnames(gtf_data) == chrom & start(gtf_data) >= start & end(gtf_data) <= end, ]

# Additionally filter for the gene of interest
gene_of_interest <- "HDAC9"
subset_gtf_data <- subset_gtf_data[subset_gtf_data$gene_name == gene_of_interest, ]

In [ ]:
# Ideogram track ----- 
gen <- "hg38"
itrack <- IdeogramTrack(genome = gen, chromosome = chrom, name = '')

displayPars(itrack) <- list(showBandId = FALSE, # Set the desired font size (10 is the default; adjust as needed)
                                col.frame = 'black', col = 'black', cex = 1.2, col.line = 'black',showTitle = TRUE,
                               fontcolor.group = 'black')  # Set the font color to black

In [ ]:
# For the transcript id plot
filtered_gtf_data <- subset_gtf_data[subset_gtf_data$type == "transcript"]
head(filtered_gtf_data)
table(filtered_gtf_data$transcript_id)
mcols(filtered_gtf_data)$feature <- filtered_gtf_data$transcript_id 
gene_track <- GeneRegionTrack(filtered_gtf_data,
                              name = "",
                              chromosome = chrom,
                              start = start, end = end,
                              transcriptAnnotation = 'feature',
                              showId = TRUE, just.group = "above",
                              fontcolour = 'black',
                              fontsize = 300
                              ) 

displayPars(gene_track) <- list(fontcolor = 'black', col = 'black', cex = 10, col.line = 'black',
                               fontcolor.group = 'black')  # Set the font color to black

In [ ]:
# For adding the exon information, let's filter the exonic information 
subset_gtf_transcripts_exons <- subset_gtf_data[subset_gtf_data$type %in% c("exon"), ]
head(subset_gtf_transcripts_exons)
subset_gtf_transcripts_exons <- subset_gtf_transcripts_exons[subset_gtf_transcripts_exons$transcript_id %in% filtered_gtf_data$transcript_id,]
table(subset_gtf_transcripts_exons$transcript_id)

exon_track <- AnnotationTrack(subset_gtf_transcripts_exons,
                              name = "Exons",
                              chromosome = chrom,
                              start = start, end = end ,
                              fill = "darkgreen",    # Color for exons
                              col = "black",
                              direction = NA)         # Border color for exons
# Ideogram track ----- 
gen <- "hg38"
itrack <- IdeogramTrack(genome = gen, chromosome = chrom, name = '')

displayPars(itrack) <- list(showBandId = FALSE, # Set the desired font size (10 is the default; adjust as needed)
                                col.frame = 'black', col = 'black', cex = 1.2, col.line = 'black',showTitle = TRUE,
                               fontcolor.group = 'black')  # Set the font color to black



In [ ]:
# For bam coverage plot: Define the bam files for tumor and normal samples

# Define the path to the BAM folder and the sample information file
bam_folder <- "."
sample_info_file <- as.data.frame(read_csv('meta_with_batchinfo.csv'))

# List all BAM files in the folder
bam_files <- list.files(path = bam_folder, pattern = "*.bam.bai", full.names = FALSE)

# Extract sample IDs from BAM filenames (the "SRRxxxxxx" part)
bam_sample_ids <- sapply(strsplit(bam_files, "_"), `[`, 1)

# Add sample IDs to a data frame with BAM file names
bam_df <- data.frame(sample_id = bam_sample_ids, bam_file = bam_files, stringsAsFactors = FALSE)


In [ ]:
colnames(sample_info_file)

sample_info_file <- sample_info_file %>%
  mutate(Tumor_Normal1 = case_when(
    Tumor_Normal %in% c("TumorTissue", "TumorTissue.2018", "TumorTissue.rep") ~ "Tumor",
    Tumor_Normal %in% c("PairedNormalTissue", "PairedNormalTissue.rep") ~ "AdjNormal"
  ))
sample_info_file <- sample_info_file[,c('Run', 'Tumor_Normal1')]

# Merge the BAM data frame with the sample information
merged_data <- merge(bam_df, sample_info_file, by.x = "sample_id", by.y = "Run")

# Filter for tumor and normal samples
library(dplyr)
library(tidyverse)

tumor_bam_files <- merged_data %>%
  filter(Tumor_Normal1 == "Tumor") %>%
  dplyr::select(bam_file)

normal_bam_files <- merged_data %>%
  filter(Tumor_Normal1 == "AdjNormal") %>%
  dplyr::select(bam_file)

dim(tumor_bam_files)
dim(normal_bam_files)

head(tumor_bam_files)
head(normal_bam_files)

In [ ]:
# Define the bam files
tnbc_bam <- tumor_bam_files$bam_file
nor_bam <- normal_bam_files$bam_file

head(tnbc_bam)
head(nor_bam)

# Remove the ".bai" extension from each filename
tnbc_bam <- gsub(".bai$", "", tnbc_bam)
nor_bam <- gsub(".bai$", "", nor_bam)

head(tnbc_bam)
head(nor_bam)

In [ ]:
# Modify to compute mean coverage
get_mean_coverage_as_granges <- function(bam_files, chromosome, start, end) {
  coverage_list <- lapply(bam_files, function(bam_file) {
    bam_data <- readGAlignments(bam_file, param=ScanBamParam(which=GRanges(chromosome, IRanges(start, end))))
    coverage(bam_data)[[chromosome]]
  })
  # Sum coverages and then divide by the number of BAM files to get the mean
  cumulative_coverage <- Reduce("+", coverage_list)
  mean_coverage <- cumulative_coverage / length(bam_files)
  
  gr <- GRanges(seqnames = chromosome,
                ranges = IRanges(start = start:(end - 1), width = 1),
                score = as.numeric(mean_coverage[start:(end - 1)]))
  return(gr)
}

# Use the new function to get mean coverage
tnbc_coverage_mean <- get_mean_coverage_as_granges(tnbc_bam, chromosome = chrom, start = start, end = end)
nor_coverage_mean <- get_mean_coverage_as_granges(nor_bam, chromosome = chrom, start = start, end = end)

In [ ]:
coverage_values <- mcols(tnbc_coverage_mean)$score
quantile(coverage_values, probs = c(0.05,0.5, 0.95))
max(coverage_values)
coverage_values <- mcols(nor_coverage_mean)$score
quantile(coverage_values, probs = c(0.05, 0.5,0.95))
max(coverage_values)

In [ ]:
# TNBC track with explicit color settings for histogram
tnbc_track <- DataTrack(range = tnbc_coverage_mean, genome = "hg38", type = "hist",
                         chromosome = chrom, start = (start), end = end,
                         name = "TNBC\nmean\ncoverage", col.histogram = 'tomato2',
                         fill.histogram='tomato2', ylim = c(0,40))

# Normal track with explicit color settings for histogram
nor_track <- DataTrack(range = nor_coverage_mean, genome = "hg38", type = "hist",
                          chromosome = chrom, start = (start), end = end,
                          name = "Normal\nmean\ncoverage", col.histogram = 'steelblue1',
                          fill.histogram='steelblue1', ylim = c(0,40))

In [ ]:
# Create GenomeAxisTrack
axis_track <- GenomeAxisTrack(name = 'Axis Track', add53 = TRUE, add35 = TRUE)
displayPars(axis_track) <- list(  # Set the desired font size (10 is the default; adjust as needed)
                                fontcolor = 'black', col = 'black', cex = 1, col.line = 'black',
                               fontcolor.group = 'black')  # Set the font color to black

# Create promoter regions (p1; p2; p3)
p1 <- AnnotationTrack(start = 18086949, end = end, chromosome = chrom,
                      name = "", fill = "pink", col = NULL, 
                     fontcolor.group = 'black',fontcolor.item = 'black')
p2 <- AnnotationTrack(start = 18495745, end = end, chromosome = chrom,
                      name = "", fill = "orange", col = NULL,
                      fontcolor.group = 'black', fontcolor.item = 'black')
p3 <- AnnotationTrack(start = 18509273, end = end, chromosome = chrom,
                      name = "", fill = "lightblue", col = NULL,
                      fontcolor.group = 'black', fontcolor.item = 'black')

In [ ]:
# For adding the H3K4me3 track for Tumor
bw.file_tum <- 'HCC1937_t2_SRX1998157.bw'
bw_tumor<- readBigwig(file=bw.file_tum,
                      chrom = "chr7",
                      chromstart = 18086949,
                      chromend = 19002416)
head(bw_tumor)
summary(bw_tumor$score)

In [ ]:
data_tum <- bw_tumor
gr_data_tumor <- GRanges(
    seqnames = data_tum$seqnames,         # Chromosome column
    ranges = IRanges(
        start = data_tum$start,        # Start positions
        end = data_tum$end             # End positions
    ),
    score = data_tum$score             # Additional metadata column
)

## Create a DataTrack object
track_tumor <- DataTrack(
    range = gr_data_tumor,
    genome = "hg38",      # Specify the genome version, e.g., "hg19" or "hg38"
    name = "HCC1937\nH3K4me3", # Name of the track
    type = "h",
    chromosome = chrom,
    ylim = c(0, 1.5),# Set limits for the y-axis
    col = "tomato2" 
)

In [ ]:
# For adding the H3K4me3 track for Normal
bw.file_nor <- 'MCF10A_t1.bw'
bw_normal<- readBigwig(file=bw.file_nor,
                      chrom = "chr7",
                      chromstart = 18086949,
                      chromend = 19002416)
head(bw_normal)
summary(bw_normal$score)

In [ ]:
data_normal <- bw_normal

gr_data_normal <- GRanges(
    seqnames = data_normal$seqnames,         # Chromosome column
    ranges = IRanges(
        start = data_normal$start,        # Start positions
        end = data_normal$end             # End positions
    ),
    score = data_normal$score             # Additional metadata column
)

## Create a DataTrack object
track_nor <- DataTrack(
    range = gr_data_normal,
    genome = "hg38",      # Specify the genome version, e.g., "hg19" or "hg38"
    name = "MCF10A\nH3K4me3",
    type = "h",  
    chromosome = chrom,
    ylim = c(0, 1.5),       # Set limits for the y-axis
    col = "steelblue1" 
)

In [ ]:
## Chunck for adding the H3K27ac enhancer chipseq data plot:------------------------->>

# For Tumor
bw.file_tum <- 'HCC1937.H3K27ac_SRX1998148.bw'
bw_tumor<- readBigwig(file=bw.file_tum,
                      chrom = "chr7",
                      chromstart = 18086949,
                      chromend = 19002416)
head(bw_tumor)
summary(bw_tumor$score)

In [ ]:
data_tum <- bw_tumor
gr_data_tumor <- GRanges(
    seqnames = data_tum$seqnames,         # Chromosome column
    ranges = IRanges(
        start = data_tum$start,        # Start positions
        end = data_tum$end             # End positions
    ),
    score = data_tum$score             # Additional metadata column
)

## Create a DataTrack object
track_tumor_enhancer <- DataTrack(
    range = gr_data_tumor,
    genome = "hg38",      # Specify the genome version, e.g., "hg19" or "hg38"
    name = "HCC1937\nH3K27ac", # Name of the track
    type = "h",  
    chromosome = chrom,
    ylim = c(0, 1.5),     # Set limits for the y-axis
    col = "tomato2" 
)

In [ ]:
# Add H3K27ac track for Normal
bw.file_nor <- 'MCF10A.H3K27ac_SRX1997949.bw'
bw_normal<- readBigwig(file=bw.file_nor,
                      chrom = "chr7",
                      chromstart = 18086949,
                      chromend = 19002416)
head(bw_normal)
summary(bw_normal$score)

In [ ]:
data_normal <- bw_normal

gr_data_normal <- GRanges(
    seqnames = data_normal$seqnames,         # Chromosome column
    ranges = IRanges(
        start = data_normal$start,        # Start positions
        end = data_normal$end             # End positions
    ),
    score = data_normal$score             # Additional metadata column
)

## Create a DataTrack object
track_nor_enhancer <- DataTrack(
    range = gr_data_normal,
    genome = "hg38",      # Specify the genome version, e.g., "hg19" or "hg38"
    name = "MCF10A\nH3K27ac",
    type = "h",  
    chromosome = chrom,
    ylim = c(0, 1.5),     # Set limits for the y-axis
    col = "steelblue1" 
)

In [ ]:
# Adding the highlight track plot for HDAC9
ht <- HighlightTrack(trackList = list(p1,p2,p3,tnbc_track,nor_track,track_tumor,track_nor,track_tumor_enhancer,track_nor_enhancer),
                    start = c(18082249,18491845,18506023), width = c(8000,8000,8000), chromosome = chrom, 
                     col = "orange", fill = "orange", inBackground = FALSE, alpha = 0.2)

In [ ]:
# Plot all tracks
pdf('may23_HDAC9_pr1078_added_Final.pdf',width = 8,height = 8)
# options(repr.plot.width = 8, repr.plot.height = 8)  # width and height in inches
plotTracks(list(axis_track, ht),
           from = 18086949, to = 18656949, chromosome = chrom,
           sizes = c(4,1,1,1,6,6,6,6,6,6),
           background.panel = "white",
           background.title = "grey39", col.axis = "white",
           extend.left = 0.025,
           cex.axis = 0.7, cex.title = 1,showTitle = TRUE)
dev.off()